In [1]:
from mendeleev import get_all_elements
from mendeleev.fetch import fetch_table
from rdkit import Chem
import pandas as pd
from tqdm import tqdm

prev_source = ['atomic_number','atomic_weight', 'atomic_radius', 'atomic_volume', 'dipole_polarizability',
                   'vdw_radius', 'en_pauling','boiling_point', 'electron_affinity', 
                   'en_allen', 'en_ghosh','mulliken_en','NumberofNeutrons','NumberofElectrons','NumberofProtons']

In [2]:
df = fetch_table('elements')

filtering_1st = ['annotation', 'description', 'electronic_configuration', 'name', 'symbol', 'jmol_color', 'cpk_color', 'molcas_gv_color', 'cas', 'geochemical_class', 'goldschmidt_class', 'discoverers', 'discovery_year', 'discovery_location', 'name_origin', 'sources', 'uses']


In [4]:
desc_filter = list(df.columns)
for target in filtering_1st:
    desc_filter.remove(target)

filtered = df[desc_filter]
filtered.columns, len(filtered.columns)

(Index(['atomic_number', 'atomic_radius', 'atomic_volume', 'block', 'density',
        'dipole_polarizability', 'electron_affinity', 'evaporation_heat',
        'fusion_heat', 'group_id', 'lattice_constant', 'lattice_structure',
        'period', 'series_id', 'specific_heat_capacity', 'thermal_conductivity',
        'vdw_radius', 'covalent_radius_cordero', 'covalent_radius_pyykko',
        'en_pauling', 'en_allen', 'proton_affinity', 'gas_basicity',
        'heat_of_formation', 'c6', 'covalent_radius_bragg', 'vdw_radius_bondi',
        'vdw_radius_truhlar', 'vdw_radius_rt', 'vdw_radius_batsanov',
        'vdw_radius_dreiding', 'vdw_radius_uff', 'vdw_radius_mm3',
        'abundance_crust', 'abundance_sea', 'en_ghosh', 'vdw_radius_alvarez',
        'c6_gb', 'atomic_weight', 'atomic_weight_uncertainty',
        'is_monoisotopic', 'is_radioactive', 'atomic_radius_rahm',
        'metallic_radius', 'metallic_radius_c12',
        'covalent_radius_pyykko_double', 'covalent_radius_pyykko_triple

In [5]:
train = pd.read_csv('/root/Competitions/DACON/4. JUMP AI 2023/data/train.csv')
train

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.259,117.37
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.172,73.47
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.585,62.45
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.475,92.60
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.337,42.43
...,...,...,...,...,...,...,...,...,...,...,...
3493,TRAIN_3493,Cn1nc(CNC(=O)Cn2nc(C(F)(F)F)c3c2CCC3)c(Cl)c1Cl,1.556,3.079,3.409,396.195,3,1,5,3.409,64.74
3494,TRAIN_3494,CCn1[nH]cc/c1=N\C(=O)c1nn(-c2ccccc2)c(=O)c2ccc...,35.560,47.630,1.912,359.381,4,1,3,1.844,77.37
3495,TRAIN_3495,CCOC(=O)CCCc1nc2cc(N)ccc2n1C,56.150,1.790,1.941,261.320,3,1,6,2.124,70.14
3496,TRAIN_3496,Nc1cc(C(=O)OCCC2CCOC2=O)cnc1Cl,0.030,2.770,0.989,284.696,5,1,5,0.989,91.51


In [6]:
def atomic_f_missing_count(target_smiles):
    atom_count = 0
    atomic_f_null_count = {}
    for smiles in tqdm(target_smiles):
        
        mol = Chem.MolFromSmiles(smiles)

        for atom in mol.GetAtoms():
            atomic_num = atom.GetAtomicNum() - 1 

            row = filtered.loc[atomic_num]
            mask = row.isna()
            missing_columns = row.index[mask]

            
            
            for missing_f in missing_columns:
                if not missing_f in atomic_f_null_count:
                    atomic_f_null_count[missing_f] = 0
                atomic_f_null_count[missing_f] += 1 

            atom_count += 1

    return atom_count, atomic_f_null_count

In [7]:
atom_count, atomic_f_null_count = atomic_f_missing_count(train.SMILES)
print(atom_count, atomic_f_null_count)

100%|██████████| 3498/3498 [00:15<00:00, 219.41it/s]

88840 {'evaporation_heat': 85657, 'fusion_heat': 85657, 'proton_affinity': 65025, 'gas_basicity': 65025, 'vdw_radius_truhlar': 88840, 'atomic_weight_uncertainty': 88136, 'is_monoisotopic': 88138, 'metallic_radius': 87423, 'metallic_radius_c12': 9685, 'lattice_constant': 695, 'vdw_radius_rt': 2, 'thermal_conductivity': 7}


In [8]:
test = pd.read_csv('/root/Competitions/DACON/4. JUMP AI 2023/data/test.csv')
t_atom_count, t_atomic_f_null_count = atomic_f_missing_count(test.SMILES)
print(t_atom_count, t_atomic_f_null_count)

100%|██████████| 483/483 [00:02<00:00, 231.77it/s]

12046 {'evaporation_heat': 11660, 'fusion_heat': 11660, 'proton_affinity': 8805, 'gas_basicity': 8805, 'vdw_radius_truhlar': 12046, 'atomic_weight_uncertainty': 11965, 'is_monoisotopic': 11965, 'metallic_radius': 11868, 'metallic_radius_c12': 1287, 'lattice_constant': 80, 'thermal_conductivity': 1, 'covalent_radius_bragg': 1, 'vdw_radius_rt': 1}


In [9]:
nan_filter = list(atomic_f_null_count.keys()) + list(t_atomic_f_null_count.keys())
nan_filter, len(nan_filter)

(['evaporation_heat',
  'fusion_heat',
  'proton_affinity',
  'gas_basicity',
  'vdw_radius_truhlar',
  'atomic_weight_uncertainty',
  'is_monoisotopic',
  'metallic_radius',
  'metallic_radius_c12',
  'lattice_constant',
  'vdw_radius_rt',
  'thermal_conductivity',
  'evaporation_heat',
  'fusion_heat',
  'proton_affinity',
  'gas_basicity',
  'vdw_radius_truhlar',
  'atomic_weight_uncertainty',
  'is_monoisotopic',
  'metallic_radius',
  'metallic_radius_c12',
  'lattice_constant',
  'thermal_conductivity',
  'covalent_radius_bragg',
  'vdw_radius_rt'],
 25)

In [10]:
filtered_2nd = list(filtered.columns)
filtered_2nd

for target in nan_filter:
    if target in filtered_2nd:
        filtered_2nd.remove(target)
    else:
        print(target)
filtered_2nd, len(filtered_2nd)

evaporation_heat
fusion_heat
proton_affinity
gas_basicity
vdw_radius_truhlar
atomic_weight_uncertainty
is_monoisotopic
metallic_radius
metallic_radius_c12
lattice_constant
thermal_conductivity
vdw_radius_rt


(['atomic_number',
  'atomic_radius',
  'atomic_volume',
  'block',
  'density',
  'dipole_polarizability',
  'electron_affinity',
  'group_id',
  'lattice_structure',
  'period',
  'series_id',
  'specific_heat_capacity',
  'vdw_radius',
  'covalent_radius_cordero',
  'covalent_radius_pyykko',
  'en_pauling',
  'en_allen',
  'heat_of_formation',
  'c6',
  'vdw_radius_bondi',
  'vdw_radius_batsanov',
  'vdw_radius_dreiding',
  'vdw_radius_uff',
  'vdw_radius_mm3',
  'abundance_crust',
  'abundance_sea',
  'en_ghosh',
  'vdw_radius_alvarez',
  'c6_gb',
  'atomic_weight',
  'is_radioactive',
  'atomic_radius_rahm',
  'covalent_radius_pyykko_double',
  'covalent_radius_pyykko_triple',
  'mendeleev_number',
  'dipole_polarizability_unc',
  'pettifor_number',
  'glawe_number',
  'molar_heat_capacity'],
 39)

In [11]:
filtered_df = df[filtered_2nd]
filtered_df = filtered_df.sort_index(axis=1)
filtered_df.to_csv('2nd_filtered.csv')

In [12]:
last_filter = [ 'abundance_crust', 'abundance_sea', 'atomic_number', 'glawe_number', 'block', 'group_id', 'period', 'pettifor_number', 'series_id', 'vdw_radius_alvarez', 'vdw_radius_batsanov', 'vdw_radius_bondi', 'vdw_radius_dreiding', 'vdw_radius_mm3', 'vdw_radius_uff', 'mendeleev_number'] 
filtered_3nd = list(filtered_df.columns)
filtered_3nd

for target in last_filter:
    if target in filtered_3nd:
        filtered_3nd.remove(target)
    else:
        print(target)
filtered_3nd, len(filtered_3nd)

(['atomic_radius',
  'atomic_radius_rahm',
  'atomic_volume',
  'atomic_weight',
  'c6',
  'c6_gb',
  'covalent_radius_cordero',
  'covalent_radius_pyykko',
  'covalent_radius_pyykko_double',
  'covalent_radius_pyykko_triple',
  'density',
  'dipole_polarizability',
  'dipole_polarizability_unc',
  'electron_affinity',
  'en_allen',
  'en_ghosh',
  'en_pauling',
  'heat_of_formation',
  'is_radioactive',
  'lattice_structure',
  'molar_heat_capacity',
  'specific_heat_capacity',
  'vdw_radius'],
 23)

In [13]:
def atom_unique(target_smiles):
    atom_count = 0
    atom_slot = {}
    for smiles in tqdm(target_smiles):
        
        mol = Chem.MolFromSmiles(smiles)

        for atom in mol.GetAtoms():
            atomic_num = atom.GetAtomicNum() - 1 
            
            if not atomic_num in atom_slot:
                atom_slot[atomic_num] = 0 
            atom_slot[atomic_num] += 1 
            
            atom_count += 1

    return atom_count, atom_slot

In [14]:
atom_count, atom_slot = atom_unique(train.SMILES)
print(atom_count)
print(len(atom_slot.keys()))

100%|██████████| 3498/3498 [00:00<00:00, 4169.96it/s]

88840
9


In [15]:
t_atom_count, t_atom_slot = atom_unique(test.SMILES)
print(t_atom_count)
print(len(t_atom_slot.keys()))

100%|██████████| 483/483 [00:00<00:00, 4016.02it/s]

12046
8


In [36]:
print(atom_slot.keys())
print(t_atom_slot.keys())
total = list(atom_slot.keys()) + list(t_atom_slot.keys())
total = list(set(total))
total

dict_keys([5, 7, 6, 15, 8, 16, 34, 33, 52])
dict_keys([5, 6, 15, 7, 16, 34, 8, 14])


[33, 34, 5, 6, 7, 8, 14, 15, 16, 52]

In [38]:
print(filtered_3nd, len(filtered_3nd))
final_df = df[filtered_3nd]
final_df

['atomic_radius', 'atomic_radius_rahm', 'atomic_volume', 'atomic_weight', 'c6', 'c6_gb', 'covalent_radius_cordero', 'covalent_radius_pyykko', 'covalent_radius_pyykko_double', 'covalent_radius_pyykko_triple', 'density', 'dipole_polarizability', 'dipole_polarizability_unc', 'electron_affinity', 'en_allen', 'en_ghosh', 'en_pauling', 'heat_of_formation', 'is_radioactive', 'lattice_structure', 'molar_heat_capacity', 'specific_heat_capacity', 'vdw_radius'] 23


,atomic_radius,atomic_radius_rahm,atomic_volume,atomic_weight,c6,c6_gb,covalent_radius_cordero,covalent_radius_pyykko,covalent_radius_pyykko_double,covalent_radius_pyykko_triple,...,electron_affinity,en_allen,en_ghosh,en_pauling,heat_of_formation,is_radioactive,lattice_structure,molar_heat_capacity,specific_heat_capacity,vdw_radius
0,25.0,154.0,14.1,1.008000,6.499027,6.51,31.0,32.0,NaN,NaN,...,0.754195,13.610,0.263800,2.20,217.998,False,HEX,28.836,14.304,110.0
1,120.0,134.0,31.8,4.002602,1.420000,1.47,28.0,46.0,NaN,NaN,...,-19.700000,24.590,0.442712,NaN,NaN,False,HEX,20.786,5.193,140.0
2,145.0,220.0,13.1,6.940000,1392.000000,1410.00,128.0,133.0,124.0,NaN,...,0.618049,5.392,0.105093,0.98,159.300,False,BCC,24.860,3.582,182.0
3,105.0,219.0,5.0,9.012183,227.000000,214.00,96.0,102.0,90.0,85.0,...,-2.400000,9.323,0.144986,1.57,324.000,False,HEX,16.443,1.825,153.0
4,85.0,205.0,4.6,10.810000,99.500000,99.20,84.0,85.0,78.0,73.0,...,0.279723,12.130,0.184886,2.04,565.000,False,TET,11.087,1.026,192.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,NaN,NaN,NaN,289.000000,NaN,NaN,NaN,143.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,True,None,NaN,NaN,NaN
114,NaN,NaN,NaN,288.000000,NaN,NaN,NaN,162.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,True,None,NaN,NaN,NaN
115,NaN,NaN,NaN,293.000000,NaN,NaN,NaN,175.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,True,None,NaN,NaN,NaN
116,NaN,NaN,NaN,294.000000,NaN,NaN,NaN,165.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,True,None,NaN,NaN,NaN


In [18]:
final_df.loc[total]

,atomic_radius,atomic_radius_rahm,atomic_volume,atomic_weight,c6,c6_gb,covalent_radius_cordero,covalent_radius_pyykko,covalent_radius_pyykko_double,covalent_radius_pyykko_triple,...,electron_affinity,en_allen,en_ghosh,en_pauling,heat_of_formation,is_radioactive,lattice_structure,molar_heat_capacity,specific_heat_capacity,vdw_radius
33,115.0,224.0,16.5,78.971000,210.00,233.0,120.0,116.0,107.0,107.0,...,2.020670,14.34,0.224033,2.55,227.200,False,HEX,25.363,0.321,190.0
34,115.0,219.0,23.5,79.904000,162.00,187.0,120.0,114.0,109.0,110.0,...,3.363588,15.88,0.241258,2.96,111.850,False,ORC,75.690,0.474,185.0
5,70.0,190.0,5.3,12.011000,46.60,47.9,73.0,75.0,67.0,60.0,...,1.262119,15.05,0.224776,2.55,716.870,False,DIA,8.517,0.709,170.0
6,65.0,179.0,17.3,14.007000,24.20,25.7,71.0,71.0,60.0,54.0,...,-1.400000,18.13,0.264930,3.04,472.440,False,HEX,29.124,1.040,155.0
7,60.0,171.0,14.0,15.999000,15.60,16.7,66.0,63.0,57.0,53.0,...,1.461113,21.36,0.304575,3.44,249.229,False,CUB,29.378,0.918,152.0
8,50.0,163.0,17.1,18.998403,9.52,10.2,57.0,64.0,59.0,53.0,...,3.401190,24.80,0.344443,3.98,79.335,False,MCL,31.304,0.824,147.0
14,100.0,223.0,17.0,30.973762,185.00,187.0,107.0,111.0,102.0,94.0,...,0.746607,13.33,0.206931,2.19,316.500,False,CUB,23.824,0.769,180.0
15,100.0,214.0,15.5,32.060000,134.00,140.0,105.0,103.0,94.0,95.0,...,2.077104,15.31,0.235960,2.58,277.170,False,ORC,22.700,0.708,180.0
16,100.0,206.0,18.7,35.450000,94.60,97.1,102.0,99.0,95.0,93.0,...,3.612725,16.97,0.263803,3.16,121.302,False,ORC,33.949,0.479,175.0
52,140.0,238.0,25.7,126.904470,385.00,389.0,139.0,133.0,129.0,125.0,...,3.059037,13.95,0.219518,2.66,106.757,False,ORC,54.430,0.214,198.0


In [34]:
def atomic_f_from_rdkit(target_smiles):
    atom_count = 0
    atom_slot = {}
    for smiles in target_smiles:
        
        mol = Chem.MolFromSmiles(smiles)

        for atom in mol.GetAtoms():
            atomic_num = atom.GetAtomicNum() - 1 

            if atomic_num in [33, 34, 5, 6, 7, 8, 14, 15, 16, 52]:
    
                if not atomic_num in atom_slot:
                    atom_slot[atomic_num] = 0 
                
                    # print(f'{atomic_num=}')
                
                    # print(f'{atom.GetBonds()=}') # rdkit.Chem.rdchem.Bond object
                    # print(f'{atom.GetHybridization()=}') # rdkit.Chem.rdchem.HybridizationType.SP3
                    # print(f'{atom.GetSmarts()=}') # String
                    print(f'{atom.GetDegree()=}') # int
                    print(f'{atom.GetTotalDegree()=}') # int
                    print(f'{atom.GetFormalCharge()=}') # int
                    print(f'{atom.GetIsAromatic()=}') # bool
                    print(f'{atom.GetNumImplicitHs()=}') # int
                    print(f'{atom.GetNumExplicitHs()=}') # int
                    print(f'{atom.GetTotalNumHs()=}') # int

                    print(f'{atom.GetNumRadicalElectrons()=}') # int
                    
                    print(f'{atom.GetImplicitValence()=}') # int
                    print(f'{atom.GetExplicitValence()=}') # int
                    print(f'{atom.GetTotalValence()=}') # int
                    print(f'{atom.IsInRing()=}') # bool
                    print()

In [52]:
# def get_atomic_features(atom_ojb):

mendeleev_atomic_f = ['atomic_radius', 'atomic_radius_rahm', 'atomic_volume', 'atomic_weight', 'c6', 'c6_gb', 
                        'covalent_radius_cordero', 'covalent_radius_pyykko', 'covalent_radius_pyykko_double', 'covalent_radius_pyykko_triple', 
                        'density', 'dipole_polarizability', 'dipole_polarizability_unc', 'electron_affinity', 'en_allen', 'en_ghosh', 'en_pauling', 
                        'heat_of_formation', 'is_radioactive', 'lattice_structure', 'molar_heat_capacity', 'specific_heat_capacity', 'vdw_radius']
mendeleev_atomic_f_table = fetch_table('elements')[mendeleev_atomic_f]

# atomic_num = atom.GetAtomicNum() -1
mendel_atom_f = []
rdkit_atom_f = []
mendel_atom_f.append(mendeleev_atomic_f_table.loc[5])



        for atom in mol.GetAtoms():

            rdkit_atom_f.append([atom.GetDegree(),
                atom.GetTotalDegree(),
                atom.GetFormalCharge(),
                atom.GetIsAromatic()*1.,
                atom.GetNumImplicitHs(),
                atom.GetNumExplicitHs(),
                atom.GetTotalNumHs(),
                atom.GetNumRadicalElectrons(),
                atom.GetImplicitValence(),
                atom.GetExplicitValence(),
                atom.GetTotalValence(),
                atom.IsInRing()*1.])
            print(temp)



[atomic_radius                        70.0
atomic_radius_rahm                  190.0
atomic_volume                         5.3
atomic_weight                      12.011
c6                                   46.6
c6_gb                                47.9
covalent_radius_cordero              73.0
covalent_radius_pyykko               75.0
covalent_radius_pyykko_double        67.0
covalent_radius_pyykko_triple        60.0
density                               2.2
dipole_polarizability                11.3
dipole_polarizability_unc             0.2
electron_affinity                1.262119
en_allen                            15.05
en_ghosh                         0.224776
en_pauling                           2.55
heat_of_formation                  716.87
is_radioactive                      False
lattice_structure                     DIA
molar_heat_capacity                 8.517
specific_heat_capacity              0.709
vdw_radius                          170.0
Name: 5, dtype: object, [1, 4, 0,

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.